In [1]:
from IPython.core.display import HTML
from IPython.lib.display import YouTubeVideo

def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()

# Intro to multiple hypothesis correction, TMM normalization, and heirarchical clustering
### Written by Reese Richardson for use in Biol Sci 378, Winter 2022, Northwestern University (rakr@u.northwestern.edu)

Welcome back to Python! Let's first import all the modules we are definitely need and a couple that we may or may not use.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# Code in Python is 'commented out' when it is preceded by a hash sign. 
# Use comments to make your code more readable!

# 2.1 Multiple Hypothesis Correction

When you are testing a single hypothesis, you can usually rely on the unadjusted p-value of your statistical test to tell you whether or not you should reject your null hypothesis. But when you are testing *many* hypotheses at once, you might have to do some correction to control both your **False Positive Rate (FPR)** and your **False Negative Rate (FNR)**. 

FPR tells you what proportion of null hypotheses are rejected among tests for which the null hypothesis is true.
FNR tells you what proportion of null hypotheses are not rejected among test for which the null hypothesis is false.

Let's model an experiment for which we are comparing the expression of `n_variables` genes between two conditions, where we have `n_replicates` replicates for each condition. For each gene, we will draw `n_replicates` observations from a normal distribution to simulate independent measures of gene expression within each condition. Out of `n_variables` null hypotheses to test, let's say that `n_alternative` null hypotheses are false. For the genes for which the null hypothesis is true, we will draw our observations from a normal distribution with $\mu = 1$ and $\sigma = 0.75$. For the `n_alternative` genes for which the null hypothesis is false, we will draw observations in Condition A from a normal distribution with $\mu = 1$ and $\sigma = 0.75$ and observations in Condition B from a normal distribution with $\mu = 2$ and $\sigma = 0.75$. Thus, when the null hypothesis is False, our observations in Condition A and our observations in Condition B will be drawn from two distributions with different underlying means. We will use Student's t-test ot compare expression in these two conditions, which assesses the difference in means between two samples.

For now, we will set `n_alternative = 0` and `n_replicates = 7`.

In [ ]:
n_variables = 10_000 # number of gens to test
n_alternative = 0 # number of null hypotheses that are false
n_replicates = 7 # number of replicates in each condition

# generate observations from Condition A
a = np.random.normal([1]*n_variables,[0.75]*n_variables, 
                     size=(n_replicates,n_variables)) 
# generate observations from Condition B
b = np.random.normal([1]*(n_variables-n_alternative) + [2]*(n_alternative),[0.75]*n_variables, 
                     size=(n_replicates,n_variables))
# generate a boolean array that tells us when we should and should not reject the null
reject_null_array = np.array([False]*(n_variables-n_alternative) + [True]*n_alternative)

# get array of p-values for each gene
p_s = scipy.stats.ttest_ind(a,b, equal_var=True).pvalue
print(p_s)

Great! The array `p_s` now stores the p-values from each of our `n_variables` individual t-tests. How many of these p-values suggest reject the null hypothesis at $\alpha = 0.05$? Let's use the sum of a boolean array to figure it out.

In [ ]:
n_p_sig = np.sum(p_s <= 0.05)

print('Total # of null hypotheses to test: ' + str(n_variables))
print('Total # of false null hypotheses: ' + str(n_alternative))
print('\n# of null hypotheses rejected by unadjusted p-value: ' + str(n_p_sig))

Yikes! Even though exactly zero null hypotheses are false, we still reject ~500 of them. That's a FPR of ~0.05! Curiously, that FPR corresponds to the value of our significance threshold $\alpha$. I wonder why that could be.

ANYWAY, the handy functions in the cell below will help us calculate FPR and FNR in these simulations. I glossed over this, but in Python, you can define functions to do repetitive tasks for you! See below.

In [ ]:
# A function definition consists of 'def', a function_name, and the input parameters. 
# In the function get_fpr, defined below, the input parameters are reject_null_array, p_s, and a sig_thresh.
# sig_thresh has a default value of 0.05 if no other value is supplied.
# note that the parameter names for a function can be equal to 
# variables we have previously defined outside of the function.

def get_fpr(reject_null_array, p_s, sig_thresh=0.05):
    fp = np.sum(~reject_null_array & (p_s <= sig_thresh))
    tn = np.sum(~reject_null_array & (p_s > sig_thresh))
    tp = np.sum(reject_null_array & (p_s <= sig_thresh))
    fn = np.sum(reject_null_array & (p_s > sig_thresh))
    return fp/(fp+tn) # no. of false positives over no. of false positives + no. of true negatives

def get_fnr(reject_null_array, p_s, sig_thresh=0.05):
    fp = np.sum(~reject_null_array & (p_s <= sig_thresh))
    tn = np.sum(~reject_null_array & (p_s > sig_thresh))
    tp = np.sum(reject_null_array & (p_s <= sig_thresh))
    fn = np.sum(reject_null_array & (p_s > sig_thresh))
    return fn/(fn+tp) # no. of false negatives over no. of false negatives + no. of true positives

Let's do another simulation where `n_alternative = 3000` (3,000 out of 10,000 of our null hypotheses are false) so we can put these functions to use.

In [ ]:
n_variables = 10_000 # number of gens to test
n_alternative = 3_000 # number of null hypotheses that are false
n_replicates = 7 # number of replicates in each condition

# generate observations from Condition A
a = np.random.normal([1]*n_variables,[0.75]*n_variables, 
                     size=(n_replicates,n_variables)) 
# generate observations from Condition B
b = np.random.normal([1]*(n_variables-n_alternative) + [2]*(n_alternative),[0.75]*n_variables, 
                     size=(n_replicates,n_variables))
# generate a boolean array that tells us when we should and should not reject the null
reject_null_array = np.array([False]*(n_variables-n_alternative) + [True]*n_alternative)

# get array of p-values for each gene
p_s = scipy.stats.ttest_ind(a,b, equal_var=True).pvalue

n_p_sig = np.sum(p_s <= 0.05)

print('Total # of null hypotheses to test: ' + str(n_variables))
print('Total # of false null hypotheses: ' + str(n_alternative))
print('\n# of null hypotheses rejected by unadjusted p-value: ' + str(n_p_sig))
print('FPR (unadjusted p): ' + str(get_fpr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))
print('FNR (unadjusted p): ' + str(get_fnr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))

Although your exact numbers will be different when you execute the cell above (because the random draws you make will be different from the random draws I make), you'll probably find that your FPR is around 5% and your FNR is around 37%. This means that we are rejecting about 5% of the null hypotheses we should be not be rejecting and not rejecting around 37% of the null hypotheses we should be rejecting. How can we correct this?

### 2.1.1 Bonferroni correction

Setting a significance threshold of $\alpha = 0.05$ ensures that, for a random individual test, the probability of rejecting a true null hypothesis is below 0.05. But what if we want to ensure that across *all* of our tests, the probability of rejecting *at least one* true null hypothesis is below 0.05 (i.e. a 95% chance that we don't get any false positives)?

This quantity (the probability of at least one false positive across all our tests is the **Family Wise Error Rate (FWER)**. There are a number of FWER-controlling procedures, but the most popular is probably Bonferroni correction. Bonferroni correction controls the FWER to be $< \alpha$ by only rejecting the null hypothesis for test $i$ with p-value $p_i$ if 

$$p_i \leq \frac{\alpha}{m}$$

where $m$ is the number of tests performed. This can also be expressed as

$$p_i * m \leq \alpha.$$

You can Bonferroni-adjust your p-values by multiplying them all by the number of tests performed and using the same significance threshold $\alpha$ (remember that these adjusted p-values should be capped at 1.0)! Let's use Bonferroni correction on the same simulation to see how our FPR improves.

In [ ]:
n_variables = 10_000 # number of gens to test
n_alternative = 3_000 # number of null hypotheses that are false
n_replicates = 7 # number of replicates in each condition

# generate observations from Condition A
a = np.random.normal([1]*n_variables,[0.75]*n_variables, 
                     size=(n_replicates,n_variables)) 
# generate observations from Condition B
b = np.random.normal([1]*(n_variables-n_alternative) + [2]*(n_alternative),[0.75]*n_variables, 
                     size=(n_replicates,n_variables))
# generate a boolean array that tells us when we should and should not reject the null
reject_null_array = np.array([False]*(n_variables-n_alternative) + [True]*n_alternative)

# get array of p-values for each gene
p_s = scipy.stats.ttest_ind(a,b, equal_var=True).pvalue

n_p_sig = np.sum(p_s <= 0.05)

# get array of Bonferroni-corrected p-values
bonferroni = p_s*n_variables
bonferroni[bonferroni > 1.0] = 1.0 # no p-values above 1.0!
n_bonferroni_sig = np.sum(bonferroni <= 0.05)

print('Total # of null hypotheses to test: ' + str(n_variables))
print('Total # of false null hypotheses: ' + str(n_alternative))
print('\n# of null hypotheses rejected by unadjusted p-value: ' + str(n_p_sig))
print('FPR (unadjusted p): ' + str(get_fpr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))
print('FNR (unadjusted p): ' + str(get_fnr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))

print('\n# of null hypotheses rejected by Bonferroni correction: ' + str(n_bonferroni_sig))
print('FPR (Bonferroni): ' + str(get_fpr(reject_null_array = reject_null_array, p_s=bonferroni, sig_thresh = 0.05)))
print('FNR (Bonferroni): ' + str(get_fnr(reject_null_array = reject_null_array, p_s=bonferroni, sig_thresh = 0.05)))

Nice! Our FPR is now very close to zero!

But wait...our FNR is now very close to one. That means we are not rejecting the vast majority of null hypotheses that are actually true--we decreased our number of Type I errors at the expense of increasing the number of Type II errors.

Bonferroni correction is actually a very conservative procedure for multiple hypothesis correction, and you should really only use it in instances where a great deal of your tests appear to reject their null hypotheses and you want to be *very* sure that you are only getting true positive results (this could be debated, but is a good rule of thumb)!

### 2.1.2 Bejamini-Hochberg correction

Let's say that you aren't too concerned about getting a sizable number of false positives, but you want to ensure that out of all your positive calls, a good number of them are true positives. Setting a significance threshold of $\alpha = 0.05$ ensures that, for a random individual test, the probability of rejecting a true null hypothesis is below 0.05. But what if we want to ensure that across *all* of our tests, the probability of rejecting *at least one* true null hypothesis is below 0.05 (i.e. a 95% chance that we don't get any false positives)?

This quantity (the number of false positive calls divided by the number of all positive calls) is the **False Discovery Rate (FDR)**. The most popular FDR-controlling method is the Benjamini-Hochberg procedure, invented in 1995 (lots of mathematics is very new!). Benjamini-Hochberg correction controls the FDR to be $< \alpha$ by only rejecting the null hypothesis for test $i$ with p-value $p_i$ if 

$$p_i \leq \frac{r_i*\alpha}{m}$$

where $m$ is the number of tests is $r_i$ is the *rank* of p-value $p_i$ when all p-values are ranked from smallest (with $ r = 1 $) to largest (where $ r = m $).

This can also be expressed as

$$\frac{p_i * m}{r_i} \leq \alpha.$$

You can Benjamini-Hochberg-adjust your p-values by multiplying them all by the number of tests performed, dividing them by their rank, and using the same significance threshold $\alpha$ (remember that these adjusted p-values should be capped at 1.0)! P-values that are adjusted to control the FDR are known as *q-values*. The q-value of any test is the minimum FDR that can be attained if that value is called significant. Let's define another function to calculate the FDR and use Benjamini-Hochberg correction on the same simulation to see how our FDR improves.

In [ ]:
def get_fdr(reject_null_array, p_s, sig_thresh=0.05):
    fp = np.sum(~reject_null_array & (p_s <= sig_thresh))
    tn = np.sum(~reject_null_array & (p_s > sig_thresh))
    tp = np.sum(reject_null_array & (p_s <= sig_thresh))
    fn = np.sum(reject_null_array & (p_s > sig_thresh))
    return fp/(fp+tp) # no. of false positives over no. of false positives + no. of true positives

In [ ]:
n_variables = 10_000 # number of gens to test
n_alternative = 3_000 # number of null hypotheses that are false
n_replicates = 7 # number of replicates in each condition

# generate observations from Condition A
a = np.random.normal([1]*n_variables,[0.75]*n_variables, 
                     size=(n_replicates,n_variables)) 
# generate observations from Condition B
b = np.random.normal([1]*(n_variables-n_alternative) + [2]*(n_alternative),[0.75]*n_variables, 
                     size=(n_replicates,n_variables))
# generate a boolean array that tells us when we should and should not reject the null
reject_null_array = np.array([False]*(n_variables-n_alternative) + [True]*n_alternative)

# get array of p-values for each gene
p_s = scipy.stats.ttest_ind(a,b, equal_var=True).pvalue

n_p_sig = np.sum(p_s <= 0.05)

# get array of Bonferroni-corrected p-values
bonferroni = p_s*n_variables
bonferroni[bonferroni > 1.0] = 1.0 # no p-values above 1.0!
n_bonferroni_sig = np.sum(bonferroni <= 0.05)

# get array of BH-corrected p-values (q-values)
bh_fdr = p_s*n_variables/(scipy.stats.rankdata(p_s))
bh_fdr[bh_fdr > 1.0] = 1.0 # no q-values above 1.0!
n_bh_fdr_sig = np.sum(bh_fdr <= 0.05)

print('Total # of null hypotheses to test: ' + str(n_variables))
print('Total # of false null hypotheses: ' + str(n_alternative))
print('\n# of null hypotheses rejected by unadjusted p-value: ' + str(n_p_sig))
print('FPR (unadjusted p): ' + str(get_fpr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))
print('FNR (unadjusted p): ' + str(get_fnr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))
print('FDR (unadjusted p): ' + str(get_fdr(reject_null_array = reject_null_array, p_s=p_s, sig_thresh = 0.05)))

print('\n# of null hypotheses rejected by BH correction: ' + str(n_bh_fdr_sig))
print('FPR (BH): ' + str(get_fpr(reject_null_array = reject_null_array, p_s=bh_fdr, sig_thresh = 0.05)))
print('FNR (BH): ' + str(get_fnr(reject_null_array = reject_null_array, p_s=bh_fdr, sig_thresh = 0.05)))
print('FDR (BH): ' + str(get_fdr(reject_null_array = reject_null_array, p_s=bh_fdr, sig_thresh = 0.05)))

Great! We've controlled our FDR. Try changing the value of `n_alternative` to see what happens to the FDR, FNR, and FPR.

It should be noted that the multiple hypothesis correction procedure one should use in their study is often dependent on one's specific interests. Are you trying to identify genes that would make good drug targets, where investigation is very labor-intensive and thus the cost of a false positive is very high? Try Bonferroni! Are you trying to identify a set of genes for which you can be fairly sure that a vast majority are actually differentially expressed? Try Benjamini-Hochberg!

# 2.2 Normalizing RNA-seq Data

It's time to finally begin working with genomics data! More specifically, we will be working with data from transcriptomics, the study of gene expression. The cells below import data from [an experiment that studied the effects of SARS-CoV-2 infection on gene expression in human lung epithelial cells (E-GEOD-147507)](https://www.ebi.ac.uk/gxa/experiments/E-GEOD-147507/Results).

In [ ]:
counts = pd.read_csv('E-GEOD-147507-raw-counts.tsv', sep='\t')
design = pd.read_csv('E-GEOD-147507-experiment-design.tsv', sep='\t')
design = design[design['Analysed'] == 'Yes'].reset_index(drop=True)

design = design[design['Sample Characteristic[cell line]'] == 'NHBE']
cols_to_norm = design['Run'].values
labels = design['Factor Value[infect]'].values
labels[labels == 'Severe acute respiratory syndrome coronavirus 2 (USA-WA1/2020)'] = 'SARS-CoV-2'
counts

The DataFrame `counts` shows each gene as a row and each sample as a column, and the values contained therein represent the number of fragments aligned to each gene for each sample. The DataFrame `design` contains metadata on each sample in each row. For instance, the column 'Sample Characteristic[cell line]' tells you what cell line was used for that sample and 'Sample Characteristic[infect] tells us what treatment each sample received. We're just interested in the runs that occured in cell line NHBE, and have stored these sample names in cols_to_norm.

Let's visualize their distribution for a given sample. We'll filter out all the genes that had no fragments aligned to them.

In [ ]:
samp_name = 'SRR11412283'
fig = plt.figure(figsize=(8,5)) # Create a 8 inch by 5 inch figure

# Define the logged bins to use for our histogram
bins = np.arange(0,np.log10(np.max(counts[samp_name])+1000),0.05)

sns.distplot(np.log10(counts[counts[samp_name] > 0][samp_name]), # plot distribution
             kde=False, # don't show the kernel density estimate
             norm_hist=False, # don't normalize the histogram (only works when kde=False)
             bins=bins) #define bins to use in histogram

plt.xlabel(xlabel=r'$log_{10}$(number of fragments aligned to gene)', fontsize=16) # set x label
plt.ylabel(ylabel='number of genes', fontsize=16) # set y label

ax = plt.gca() # get current axis (gca) and store it in 'ax'

ax.tick_params(labelsize=14) # set the fontsize of the tick labels

# some fancy line stuff
for pos in ['bottom', 'left']:
    ax.spines[pos].set_linewidth(1.25)
    ax.spines[pos].set_color('k') #black
for pos in ['top', 'right']:
    ax.spines[pos].set_visible(False) # turn off top and right side of frame
    
plt.grid(axis='y') # add grid lines to the y axis only

In [ ]:
design = design[design['Sample Characteristic[cell line]'] == 'NHBE']
cols_to_norm = design['Run'].values
labels = design['Factor Value[infect]'].values
labels[labels == 'Severe acute respiratory syndrome coronavirus 2 (USA-WA1/2020)'] = 'SARS-CoV-2'

Here, we can see that many genes have very few counts and few genes have very many counts. Most genes have been 10 and 1000 counts. More than 40,000 genes didn't have a single fragment aligned in this sample! If we sort the genes in our sample from least to most fragments and then take a cumulative sum...

In [ ]:
fig = plt.figure(figsize=(6,5)) # Create a 6 inch by 5 inch figure

samp_name = 'SRR11412277'
#Taking the cumulative sum of counts
cumulative_sum_counts_slice = counts[counts[samp_name] > 0][samp_name].sort_values().cumsum()
plt.plot(np.arange(0, len(cumulative_sum_counts_slice))/len(cumulative_sum_counts_slice), #normalized rank on x
         cumulative_sum_counts_slice, linewidth=2, label=samp_name) # cumulative sum of counts on y

plt.xlabel('normalized gene rank', fontsize=16)
plt.ylabel('number of fragments', fontsize=16)
ax = plt.gca() # get current axis (gca) and store it in 'ax'

ax.tick_params(labelsize=14) # set the fontsize of the tick labels
ax.set_ylim(bottom=0)
ax.set_xlim([0,1.02])
ax.legend(fontsize=16, frameon=False)
# some fancy line stuff
for pos in ['bottom', 'left']:
    ax.spines[pos].set_linewidth(1.25)
    ax.spines[pos].set_color('k') #black
for pos in ['top', 'right']:
    ax.spines[pos].set_visible(False) # turn off top and right side of frame
    
plt.grid(axis='y') # add grid lines to the y axis only

We see that the top 20% of genes absorb more than 80% of this sample's 1,000,000 fragments. 

In class, we talked about why we would want to normalize for library size when making between-sample comparisons. The most popular way to normalize for library size is by dividing each gene's count by the sum of all counts. Let's show an example of two samples in our experiment where this kind of normalization might not be enough.

In [ ]:
fig = plt.figure(figsize=(6,5)) # Create a 6 inch by 5 inch figure

samp_name = 'SRR11412277'
cumulative_sum_counts_slice = counts[counts[samp_name] > 0][samp_name].sort_values().cumsum()
plt.plot(np.arange(0, len(cumulative_sum_counts_slice))/len(cumulative_sum_counts_slice), #normalized rank on x
         cumulative_sum_counts_slice, linewidth=2, label=samp_name) # cumulative sum of counts on y

samp_name = 'SRR11412280'
cumulative_sum_counts_slice = counts[counts[samp_name] > 0][samp_name].sort_values().cumsum()
plt.plot(np.arange(0, len(cumulative_sum_counts_slice))/len(cumulative_sum_counts_slice), #normalized rank on x
         cumulative_sum_counts_slice, linewidth=2, label=samp_name) # cumulative sum of counts on y

plt.xlabel('normalized gene rank', fontsize=16)
plt.ylabel('number of fragments', fontsize=16)
ax = plt.gca() # get current axis (gca) and store it in 'ax'

ax.tick_params(labelsize=14) # set the fontsize of the tick labels
ax.set_ylim(bottom=0)
ax.set_xlim([0,1.02])
ax.legend(fontsize=16, frameon=False)

ax.tick_params(labelsize=14) # set the fontsize of the tick labels
ax.set_ylim(bottom=0)
ax.set_xlim([0,1.02])
# some fancy line stuff
for pos in ['bottom', 'left']:
    ax.spines[pos].set_linewidth(1.25)
    ax.spines[pos].set_color('k') #black
for pos in ['top', 'right']:
    ax.spines[pos].set_visible(False) # turn off top and right side of frame
    
plt.grid(axis='y') # add grid lines to the y axis only

Interesting! For the lowest 99ish percent of genes, the second sample has a greater cumulative number of counts. But suddenly, once we include the genes with the absolute highest number of fragments, the first sample surpasses the second sample. When comparing two samples with different library sizes, it might be the case that the majority of that difference is attributable to a small subset of highly-expressed genes. **Trimmed mean of m-values (TMM) normalization** is design to correct for these situations when performing between-sample comparison.

I won't get into the weeds on how exactly TMM normalization works, but I highly encourage you to read [the paper that put it forward](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2010-11-3-r25). It is surprisingly easy to follow and lays out the case for TMM normalization quite nicely! The cell below defines some functions that make tmm normalization easy.

In [ ]:
def get_tmm(counts, ref_col, test_col, trim_m=0.30, trim_a=0.05):

    '''
        Calculates TMM value between ref_col and test_col
        
        counts = count matrix (pd.DataFrame)
        ref_col = reference column (str)
        test_col = test columns (str)
        trim_m = extremes to trim when taking trimmed mean of M values (float)
        trim_a = extremes to trim when taking trimmed mean of M values (float)
        
    '''

    if ref_col == test_col:
        counts_slim = counts[(counts[ref_col].values > 0) & (counts[test_col].values > 0)][[ref_col]]
    else:
        counts_slim = counts[(counts[ref_col].values > 0) & (counts[test_col].values > 0)][[ref_col, test_col]]

    n_k = counts_slim[test_col].sum()
    n_r = counts_slim[ref_col].sum()

    m_k = np.log2(counts_slim[test_col].values/n_k)-np.log2(counts_slim[ref_col].values/n_r)
    a_k = 0.5*np.log2((counts_slim[test_col].values/n_k)*(counts_slim[ref_col].values/n_r))
    w_k = (n_k - counts_slim[test_col].values)/(n_k*counts_slim[test_col].values) + (n_r - counts_slim[ref_col].values)/(n_r*counts_slim[ref_col].values)

    trim_array_m = (m_k <= np.percentile(m_k, 100*(1-(trim_m/2)))) & ((m_k >= np.percentile(m_k, 100*((trim_m/2)))))
    trim_array_a = (a_k <= np.percentile(a_k, 100*(1-(trim_a/2)))) & ((a_k >= np.percentile(a_k, 100*((trim_a/2)))))

    m_k = m_k[trim_array_m & trim_array_a]
    a_k = a_k[trim_array_m & trim_array_a]
    w_k = w_k[trim_array_m & trim_array_a]
    tmm_k = 2**(np.sum(w_k*m_k)/np.sum(w_k))
    return tmm_k

def get_m_a(counts, ref_col, test_col, trim_m=0.30, trim_a=0.05):

    '''
        Calculates TMM value between ref_col and test_col
        
        counts = count matrix (pd.DataFrame)
        ref_col = reference column (str)
        test_col = test columns (str)
        trim_m = extremes to trim when taking trimmed mean of M values (float)
        trim_a = extremes to trim when taking trimmed mean of M values (float)
        
    '''

    if ref_col == test_col:
        counts_slim = counts[(counts[ref_col].values > 0) & (counts[test_col].values > 0)][[ref_col]]
    else:
        counts_slim = counts[(counts[ref_col].values > 0) & (counts[test_col].values > 0)][[ref_col, test_col]]

    n_k = counts_slim[test_col].sum()
    n_r = counts_slim[ref_col].sum()

    m_k = np.log2(counts_slim[test_col].values/n_k)-np.log2(counts_slim[ref_col].values/n_r)
    a_k = 0.5*np.log2((counts_slim[test_col].values/n_k)*(counts_slim[ref_col].values/n_r))
    return m_k, a_k

def norm_tmm(counts, columns_to_norm, ref_col=None):
    '''
        Normalizes count matrix by TMM.
        
        counts = count matrix (pd.DataFrame)
        ref_col = reference column (str)
        columns_to_norm = columns to be normed (str)
        
    '''
    if ref_col == None:
        ref_col = columns_to_norm[0]
    counts[counts.columns[~counts.columns.isin(cols_to_norm)]]
        
    tmm_array = []
    for col in columns_to_norm:
        tmm_array.append(get_tmm(counts = counts, ref_col=ref_col, test_col=col))
    tmm_array = np.array(tmm_array)

    norm_counts = counts[columns_to_norm]/(np.sqrt(tmm_array)*counts[columns_to_norm].sum(axis=0))
    norm_counts[counts.columns[~counts.columns.isin(columns_to_norm)]] = counts[counts.columns[~counts.columns.isin(columns_to_norm)]]
    return norm_counts

To demonstrate the effect of TMM, let's use samples SRR11412220 and SRR11412223 and create some MA plots before and after TMM normalization (MA plots show the average expression of a gene between two samples on the x-axis and the fold-change of that gene between those two samples on the y-axis. You'll see them a lot in transcriptomics).

In [ ]:
test_col = 'SRR11412220'
ref_col = 'SRR11412223'

counts_norm = counts.copy() # create a copy of our counts DataFrame
fig = plt.figure(figsize=(6,5)) # Create a 6 inch by 5 inch figure

plt.axhline(0, color='k')
m, a = get_m_a(counts_norm, test_col=test_col, ref_col=ref_col)
plt.scatter(a, m, alpha=0.05, linewidth=0, s=20)

plt.ylabel(ylabel=r'$log_{2}$(fold change)', fontsize=16) # set y label
plt.xlabel(xlabel=r'average $log_{2}$(expression)', fontsize=16) # set x label

ax = plt.gca() # get current axis (gca) and store it in 'ax'
ax.tick_params(labelsize=14) # set the fontsize of the tick labels
ax.set_ylim([-7,7])
# some fancy line stuff

for pos in ['bottom', 'left']:
    ax.spines[pos].set_linewidth(1.25)
    ax.spines[pos].set_color('k') #black
for pos in ['top', 'right']:
    ax.spines[pos].set_visible(False) # turn off top and right side of frame

Notice that before normalization, the genes in each sample are not centered around a fold-change of zero, as we would expect in a comparison where the great majority of genes are not actually differentially expressed. This effect is especially pronounced for lowly-expressed genes. Let's normalize and see what happens.

In [ ]:
test_col = 'SRR11412220'
ref_col = 'SRR11412223'

# getting a normalized count matrix
counts_norm = norm_tmm(counts.copy(), columns_to_norm=cols_to_norm, ref_col=cols_to_norm[0])
fig = plt.figure(figsize=(6,5)) # Create a 6 inch by 5 inch figure

plt.axhline(0, color='k')
m, a = get_m_a(counts_norm, test_col=test_col, ref_col=ref_col)
plt.scatter(a, m, alpha=0.05, linewidth=0, s=20)

plt.ylabel(ylabel=r'$log_{2}$(fold change)', fontsize=16) # set y label
plt.xlabel(xlabel=r'average $log_{2}$(expression)', fontsize=16) # set x label

ax = plt.gca() # get current axis (gca) and store it in 'ax'
ax.tick_params(labelsize=14) # set the fontsize of the tick labels
ax.set_ylim([-7,7])
# some fancy line stuff

for pos in ['bottom', 'left']:
    ax.spines[pos].set_linewidth(1.25)
    ax.spines[pos].set_color('k') #black
for pos in ['top', 'right']:
    ax.spines[pos].set_visible(False) # turn off top and right side of frame

Looks much better!

TMM normalization normalizes library sizes in reference to a reference sample. You'll notice that this reference column is passed as a parameter `ref_col` to the function `norm_tmm`. No matter which sample you choose as a reference, you will get the same TMM normalization every time.

You'll notice that TMM does not normalize for gene length, GC content, or any other gene property. That's because TMM is designed for between-sample comparison, and we assume that these parameters remain the same between samples (there are some issues with this assumption, but let's not dive down that rabbit hole right now). If you want to perform within-sample comparison (for instance, determining the 10 most abundant mRNAs in our cell lines), it would be necessary to normalize for these parameters. [GeTMM](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2246-7) is a version of TMM that normalizes for gene length, and is thus appropriate for within-sample and between-sample comparison.

Let's demonstrate the effectiveness of TMM normalization by performing heirarchical clustering on our data with and without normalization!

# 2.3 Heirarchical clustering

### 2.3.1 SARS-CoV-2 infection in normal human bronchial epithelial cells (without normalization)

Let's say that, among genes that are expressed differently in human bronchial epithelial cells, we are interested in finding out which genes have similar patterns of expression. We'll re-import our data (just in case you skipped to this section)...

In [ ]:
counts = pd.read_csv('E-GEOD-147507-raw-counts.tsv', sep='\t')
design = pd.read_csv('E-GEOD-147507-experiment-design.tsv', sep='\t')
design = design[design['Analysed'] == 'Yes'].reset_index(drop=True)

design = design[design['Sample Characteristic[cell line]'] == 'NHBE']
cols_to_norm = design['Run'].values
labels = design['Factor Value[infect]'].values
labels[labels == 'Severe acute respiratory syndrome coronavirus 2 (USA-WA1/2020)'] = 'SARS-CoV-2'

...and then perform a t-test to see which genes have a different mean between the mock-infected cells and the SARS-CoV-2-infected cells. We'll also perform Bonferroni and Benjamini-Hochberg correction of the p-values derived from these t-tests to try to find a suitable group of highly-variable genes to analyze.

(Note that t-tests are rarely ever appropriate for gene expression data, and most softwares for differential expression analysis use more sophisticated statistical tests. Because we don't want to get too involved, we'll refrain from using these tests for now.)

In [ ]:
counts_norm = counts.copy()

# Let's perform a t-test to compare our two groups' means.
# Notice that we've set equal_var to False. 
# Can we safely assume that the variances in our two populations are equal?
a = counts_norm[cols_to_norm].values[:,labels == 'mock']
b = counts_norm[cols_to_norm].values[:,labels == 'SARS-CoV-2']
counts_norm['ttest_p'] = scipy.stats.ttest_ind(a.T,b.T, equal_var=False).pvalue
counts_norm = counts_norm.dropna(subset=['ttest_p'])

# Bonferroni correction
counts_norm['bonferroni_fwer'] = counts_norm['ttest_p']*len(counts_norm['ttest_p'])
counts_norm['bonferroni_fwer'].loc[counts_norm['bonferroni_fwer'] > 1] = 1.0

# BH-correction
counts_norm['bh_fdr'] = counts_norm['ttest_p']*len(counts_norm['ttest_p'])/counts_norm['ttest_p'].rank()
counts_norm['bh_fdr'].loc[counts_norm['bh_fdr'] > 1] = 1.0

It is always a good idea to plot the distributions of your p-values! For heirarchical clustering, this will help us select a group of highly-variable genes to include.

In [ ]:
bins = np.arange(0, 1.00001, 0.01)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(5,6))

sns.distplot(counts_norm['ttest_p'], ax=axes[0], kde=False, norm_hist=False, bins=bins)
axes[0].set_xlabel('unadjusted p-values', fontsize=16)

sns.distplot(counts_norm['bonferroni_fwer'], ax=axes[1], kde=False, norm_hist=False, bins=bins)
axes[1].set_xlabel('Bonferroni-adjusted p-values', fontsize=16)

sns.distplot(counts_norm['bh_fdr'], ax=axes[2], kde=False, norm_hist=False, bins=bins)
axes[2].set_xlabel('BH-adjusted p-values', fontsize=16)

for ax in axes:
    ax.set_xlim([-0.005,1.005])
    ax.tick_params(labelsize=14) # set the fontsize of the tick labels
    ax.axvline(0.05, color=sns.color_palette()[3], linestyle='dashed')
    
fig.tight_layout()

Well, it looks like without normalization, very few genes are found as differentially expressed (DE) with an unadjusted p-value are none are found as DE with Bonferroni or BH correction. For the purpose of demonstration, let's proceed by clustering the genes that pass ttest_p < 0.05.

In [ ]:
counts_norm_slim = counts_norm[counts_norm['ttest_p'] < 0.05] # grab our set of genes
# set index to gene name and only take normalized count columnss
counts_norm_slim = counts_norm_slim.set_index('Gene Name')[cols_to_norm]
counts_norm_slim.columns = labels # rename our samples by which condition they represent

g = sns.clustermap(counts_norm_slim, # data to cluster
                   method='ward', # method to use for heiarchical clustering
                   z_score=0, # which axis to calculate the z-score on (0 = row axis)
                   col_cluster=True, # cluster the columns
                   vmin=-2.5, vmax=2.5, # minimum and maximum values of our colobar
                   cmap='seismic_r') # which palette we want to use for our colorbar

Congratulations! You've performed your first heirarchical clustering. The color of each cell represents the z-score of that gene within that row. Thus, blue cells represent genes that are expressed above the mean for that row, and red cells represent genes that are clustered below the mean for that row. Some people will prefer to show this data in green and red (a relic of the days of microarray analysis).

Notice that we've clustered both genes (rows) and samples (cols). Clustering samples is fine to do as long as you understand that this clustering groups samples by their behavior among this somewhat arbitrarily-selected group of genes, with equal weight applied to each gene. If we wanted to make a broader statement about similarity of gene expression between samples, you would need to perform within-sample normalization and would probably want to perform a Pearson correlation between all genes in your samples.

Although this plot looks very nice and appropriately clusters our samples by treatment (see column labels on the bottom of our graph), there is something off--for a group of four of our SARS-CoV-2-infected samples, there is a group of very highly-expressed genes that does not appear in our other samples. In fact, when you look at the dendrogram at the top of the graph, you can see that this group of samples does not cluster with the other SARS-CoV-2 samples. These are artefacts that demonstrate the need for normalization.

### 2.3.1 SARS-CoV-2 infection in normal human bronchial epithelial cells (with TMM normalization)

Let's start over, but this time normalizing our data with TMM.

In [ ]:
counts_norm = counts.copy()
counts_norm = norm_tmm(counts_norm, columns_to_norm=cols_to_norm, ref_col=cols_to_norm[0])

# Let's perform a t-test to compare our two groups' means.
# Notice that we've set equal_var to False. 
# Can we safely assume that the variances in our two populations are equal?
a = counts_norm[cols_to_norm].values[:,labels == 'mock']
b = counts_norm[cols_to_norm].values[:,labels == 'SARS-CoV-2']
counts_norm['ttest_p'] = scipy.stats.ttest_ind(a.T,b.T, equal_var=False).pvalue
counts_norm = counts_norm.dropna(subset=['ttest_p'])

# Bonferroni correction
counts_norm['bonferroni_fwer'] = counts_norm['ttest_p']*len(counts_norm['ttest_p'])
counts_norm['bonferroni_fwer'].loc[counts_norm['bonferroni_fwer'] > 1] = 1.0

# BH-correction
counts_norm['bh_fdr'] = counts_norm['ttest_p']*len(counts_norm['ttest_p'])/counts_norm['ttest_p'].rank()
counts_norm['bh_fdr'].loc[counts_norm['bh_fdr'] > 1] = 1.0

In [ ]:
bins = np.arange(0, 1.00001, 0.01)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(5,6))

sns.distplot(counts_norm['ttest_p'], ax=axes[0], kde=False, norm_hist=False, bins=bins)
axes[0].set_xlabel('unadjusted p-values', fontsize=16)

sns.distplot(counts_norm['bonferroni_fwer'], ax=axes[1], kde=False, norm_hist=False, bins=bins)
axes[1].set_xlabel('Bonferroni-adjusted p-values', fontsize=16)

sns.distplot(counts_norm['bh_fdr'], ax=axes[2], kde=False, norm_hist=False, bins=bins)
axes[2].set_xlabel('BH-adjusted p-values', fontsize=16)

for ax in axes:
    ax.set_xlim([-0.005,1.005])
    ax.tick_params(labelsize=14) # set the fontsize of the tick labels
    ax.axvline(0.05, color=sns.color_palette()[3], linestyle='dashed')
    
fig.tight_layout()

Right off the bat, you can see that our p-value distributions make a lot more sense. This time, we'll set a signifcance threshold of q < 0.05.

In [ ]:
counts_norm_slim = counts_norm[counts_norm['bh_fdr'] < 0.05]
counts_norm_slim = counts_norm_slim.set_index('Gene Name')[cols_to_norm]
counts_norm_slim.columns = labels

g = sns.clustermap(counts_norm_slim, # data to cluster
                   method='ward', # method to use for heiarchical clustering
                   z_score=0, # which axis to calculate the z-score on (0 = row axis)
                   col_cluster=True, # cluster the columns
                   vmin=-2.5, vmax=2.5, # minimum and maximum values of our colobar
                   cmap='seismic_r') # which palette we want to use for our colorbar

Would you look at that! We've create a pretty nice-looking cluster plot.

From the gene dendrogram (on the left), you can see that there are two large clusters--genes that are upregulated in SARS-CoV-2 infection and genes that are upregulated in mock infection. There are still some outlying genes, but normalization has scaled back their influence. You'll also notice from the sample dendrogram that all our SARS-CoV-2 samples and mock samples cluster amongst themselves.

But here's the real kicker: we've also vizualized an un-annotated underlying structure to this data. You might have noticed that the lowest-level clusters formed in our sample dendrogram are *all* groups of four. It appears that the researchers that generated this data by processed four samples at a time! It is also possible that we are looking at groups of four technical replicates among three biological replicates.

Gene expression patterns are highly subject to change and even minute differences in preparation can generate wildly different gene expression signatures, especially in cell cultures, as shown above. This is why having a large number of replicates is of paramount importance in transcriptomics experiments.

The previous example is from an experiment comparing two conditions. However, most of the time, you'll only want to perform clustering on experiments comparing *more than two conditions*.

### 2.3.2 Esophageal adenocarcinoma and Barrett's esophagus

[Esophageal adenocarcinoma (EAC)](https://www.mayoclinic.org/diseases-conditions/esophageal-cancer/symptoms-causes/syc-20356084) is a very common type of cancer, most commonly brought about by smoking. It is often preceded by a condition called Barrett's esophagus, wherein chronic acid reflux gradually causes a change in the type of cells that line the lower esophagus where it meets the stomach.

We're going to work with data from an experiment [(E-MTAB-4054)](https://www.ebi.ac.uk/gxa/experiments/E-MTAB-4054/Results) where researchers collected tissue samples from patients demonstrating normal esophagus (NORM), non-dysplastic Barrett's esophagus (NDBE), Barrett's esophagus with low-grade dysplasia (LGD), and esophageal adenocarcinoma (EAC). Feel free to explore the `counts` and `design` DataFrames below.

In [ ]:
counts = pd.read_csv('E-MTAB-4054-raw-counts.tsv', sep='\t')
design = pd.read_csv('E-MTAB-4054-experiment-design.tsv', sep='\t')
design = design[design['Analysed'] == 'Yes'].reset_index(drop=True)

design['labels'] = ['']*len(design)

design['labels'].loc[(design[['Factor Value[disease]', 'Factor Value[disease staging]']].values == ["Barrett's esophagus", "non-dysplastic"]).all(axis=1)] = 'NDBE'
design['labels'].loc[(design[['Factor Value[disease]', 'Factor Value[disease staging]']].values == ["Barrett's esophagus", "low-grade dysplasia"]).all(axis=1)] = 'LGD'
design['labels'].loc[(design['Factor Value[disease]'].values == "esophageal adenocarcinoma")] = 'EAC'
design['labels'].loc[(design['Factor Value[disease]'].values == "normal")] = 'NORM'

design = design.sort_values('labels')
cols_to_norm = design['Run'].values
labels = design['labels'].values

Instead of performing a t-test to compare two groups, we'll use a one-way ANOVA to compare these four groups. The cell below normalizes our counts, performs the ANOVA test on each row, applies multiple hypothesis correction, and plots our p- and q-values. You'll notice that the p-value column is still labeled ttest_p. This is because I am lazy and don't want to change it.

In [ ]:
counts_norm = counts.copy()

counts_norm = norm_tmm(counts_norm, columns_to_norm=cols_to_norm, ref_col=cols_to_norm[0])

a = counts_norm[cols_to_norm].values[:,labels == 'EAC']
b = counts_norm[cols_to_norm].values[:,labels == 'LGD']
c = counts_norm[cols_to_norm].values[:,labels == 'NDBE']
d = counts_norm[cols_to_norm].values[:,labels == 'NORM']
counts_norm['ttest_p'] = scipy.stats.f_oneway(a.T, b.T, c.T, d.T).pvalue
counts_norm = counts_norm.dropna(subset=['ttest_p'])
counts_norm['bonferroni_fwer'] = counts_norm['ttest_p']*len(counts_norm['ttest_p'])
counts_norm['bonferroni_fwer'][counts_norm['bonferroni_fwer'] > 1] = 1.0
counts_norm['bh_fdr'] = counts_norm['ttest_p']*len(counts_norm['ttest_p'])/counts_norm['ttest_p'].rank()
counts_norm['bh_fdr'][counts_norm['bh_fdr'] > 1] = 1.0

In [ ]:
bins = np.arange(0, 1.00001, 0.01)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(5,6))

sns.distplot(counts_norm['ttest_p'], ax=axes[0], kde=False, norm_hist=False, bins=bins)
axes[0].set_xlabel('unadjusted p-values', fontsize=16)

sns.distplot(counts_norm['bonferroni_fwer'], ax=axes[1], kde=False, norm_hist=False, bins=bins)
axes[1].set_xlabel('Bonferroni-adjusted p-values', fontsize=16)

sns.distplot(counts_norm['bh_fdr'], ax=axes[2], kde=False, norm_hist=False, bins=bins)
axes[2].set_xlabel('BH-adjusted p-values', fontsize=16)

for ax in axes:
    ax.set_xlim([-0.005,1.005])
    ax.tick_params(labelsize=14) # set the fontsize of the tick labels
    ax.axvline(0.05, color=sns.color_palette()[3], linestyle='dashed')
    
fig.tight_layout()

Whoa! These p-value distributions look very different from the previous case. Because adding more conditions to an experiment will inevitably increase the number of genes that are found as differentially expressed in at least one condition, you will probably find that lots of genes change their expression across conditions with an ANOVA test. Further, cancer tends to completely upend gene expression in a tissue, so it's natural that we might find many genes as significantly differentially expressed between conditions.

So that we don't spend all day waiting, let's just select the 2000 genes with the lowest q-value to proceed with clustering. I've also added a few lines of code that will add a color to each of our samples based on what category it falls into, which will help with visual clarity.

In [ ]:
counts_norm_slim = counts_norm[counts_norm['bh_fdr'].rank() <= 2000]
counts_norm_slim = counts_norm_slim.set_index('Gene Name')[cols_to_norm]
counts_norm_slim.columns = labels

# Define colors to label our columns to make the diagnosis categories of our clustermap easier to read
lut = dict(zip(set(labels), ['k', 'r', 'g', 'b'])) # black, red, green, blue
col_colors = pd.DataFrame(labels)[0].map(lut)

g = sns.clustermap(counts_norm_slim, # data to cluster
                   method='ward', # method for heiarchical clustering
                   col_cluster=True, # cluster columns
                   z_score=0, # axis to perform z standardization
                   vmin=-2.5, vmax=2.5, # upper and lower limits of our colorbar
                   cmap='seismic_r', # palette to use for our colorbar
                   col_colors=[col_colors]) # what colors to label our columns on the margin

# define legend for col colors
ax = g.ax_heatmap
handles1 = [mpl.patches.Patch(facecolor=lut[name]) for name in lut]

leg1 = g.ax_row_dendrogram.legend(handles1, lut, title='diagnosis',
           bbox_to_anchor=(0.97, 0.70), bbox_transform=plt.gcf().transFigure, loc='upper left', frameon=False, fontsize=16)
plt.setp(leg1.get_title(),fontsize=16)
leg1._legend_box.align = "left"

This clustermap yields some very exciting insights! For one, it appears that there is a large cluster of genes that is downregulated in Barrett's esophagus and beyond. On the other hand, there is also a large cluster of genes that is upregulated in Barrett's esophagus, including a sub-cluster of genes that are uniquely highly-expressed in EAC. As we might expect, among these genes the expression profiles of our NORM samples cluster nicely together, as well as the EAC samples. NDBE and LGD, both non-malignant stages Barrett's esophagus, seem to intermingle some. You'll also notice an outlier NDBE sample that clusters among the NORM samples. Could this indicate a misdiagnosis?

While many researchers believe that RNA-seq could one day be used for sensitive clinical diagnostics, let's not get ahead of ourselves. In all likelihood, this outlier likely represents an edge case between normal esophagus and non-dysplastic Barrett's esophagus. Still, the distinct transcriptional profiles revealed by RNA-seq between different diagnoses speak to its promise for diagnostic medicine.

(One of the main challenges that keeps RNA-seq from being employed at the bedside is a lack of reproducibility, but that's a story for another time).

# Congratulations!
## We've reached the end of our second lesson!

Thus far, we've mostly just made pretty pictures. Next week, we'll start extracting meaning from which genes fall in which clusters. 

To practice the skills you've acquired thus far, complete the excercises below.

#### Exercise 2.1

Using code from Section 2.1, demonstrate what happens to the number of null hypotheses rejected by each method for multiple hypothesis correction when you increase or decrease the number of replicates used in your experiment.

#### Exercise 2.2
The cell below imports and normalizes data from [RNA-seq of the ileum of pediatric Crohn's patients and non-inflammatory controls (E-GEOD-101794)](https://www.ebi.ac.uk/gxa/experiments/E-GEOD-101794/Results). The labels 'A1a' and 'A1b' refer to the [Paris classification](https://www.naspghan.org/files/documents/pdfs/training/curriculum-resources/ibd/Paris_Classification.pdf) (i.e. the age of diagnosis). 

FYI: There are 304 samples in this dataset, so everything might take a bit longer to run. 

The column 'anova_p' shows the p-value returned when an ANOVA test is performed on each row (changing the column name was actually pretty painless. Let this be a lesson!). At a threshold of 0.05, how many null hypotheses are rejected using
1. the unadjusted p-value?
2. Bonferroni correction?
3. Benjamini-Hochberg FDR correction?

In [ ]:
counts = pd.read_csv('E-GEOD-101794-raw-counts.tsv', sep='\t')
design = pd.read_csv('E-GEOD-101794-experiment-design.tsv', sep='\t')
design = design[design['Analysed'] == 'Yes'].reset_index(drop=True)

#design = design[design['Sample Characteristic[cell line]'] == 'A549']

design['labels'] = ['']*len(design)

design['labels'].loc[(design[['Sample Characteristic[clinical information]', 'Sample Characteristic[disease]']].values == ["A1a Paris classification", "Crohn's disease"]).all(axis=1)] = 'CROHN A1a'
design['labels'].loc[(design[['Sample Characteristic[clinical information]', 'Sample Characteristic[disease]']].values == ["A1a Paris classification", "non inflammatory bowel disease control"]).all(axis=1)] = 'CONTROL A1a'
design['labels'].loc[(design[['Sample Characteristic[clinical information]', 'Sample Characteristic[disease]']].values == ["A1b Paris classification", "Crohn's disease"]).all(axis=1)] = 'CROHN A1b'
design['labels'].loc[(design[['Sample Characteristic[clinical information]', 'Sample Characteristic[disease]']].values == ["A1b Paris classification", "non inflammatory bowel disease control"]).all(axis=1)] = 'CONTROL A1b'
design['labels'].loc[(design['Sample Characteristic[clinical information]'].values == "normal")] = 'NORM'

design = design.sort_values('labels')
cols_to_norm = design['Run'].values
labels = design['labels'].values

In [ ]:
counts_norm = counts.copy()

counts_norm = norm_tmm(counts_norm, columns_to_norm=cols_to_norm, ref_col=cols_to_norm[0])
counts_norm = counts_norm[counts_norm[cols_to_norm].sum(axis=1) > 0]

a = counts_norm[cols_to_norm].values[:,labels == np.unique(labels)[0]]
b = counts_norm[cols_to_norm].values[:,labels == np.unique(labels)[1]]
c = counts_norm[cols_to_norm].values[:,labels == np.unique(labels)[2]]
d = counts_norm[cols_to_norm].values[:,labels == np.unique(labels)[3]]
counts_norm['anova_p'] = scipy.stats.f_oneway(a.T, b.T, c.T, d.T).pvalue
counts_norm = counts_norm.dropna(subset=['anova_p'])

#### Exercise 2.3
Cluster the data from Exercise 2.2! Use an inclusion criterion that makes sense, but won't leave you sitting at the computer for 45 minutes waiting.